In [2]:
import pandas as pd
import numpy as np

# Transformar o Float64 >> para melhor visualização
pd.set_option('float_format','{:,.2f}'.format)

# Retirar Criticas
pd.set_option('mode.chained_assignment', None)

In [63]:
itens_patrim = pd.read_excel('Base_Itens_Patrimoniais.xlsx', header=3, dtype=object)
itens_patrim.drop(itens_patrim.tail(3).index, inplace=True)
itens_patrim.dropna(subset=['IMPLICA Despesa'], inplace=True)
df_servicos = itens_patrim[itens_patrim['Tipo'].str.contains('Serviço', case=False, na=False)]
df_servicos = df_servicos[~df_servicos['Nome'].str.contains('taxa', case=False, na=False)]
df_servicos = df_servicos[~df_servicos['Nome'].str.contains('utilizar', case=False, na=False)]
df_servicos.drop(['SE Despesa', 'SE Receita', 'IMPLICA Receita'], axis=1, inplace=True)
df_servicos = df_servicos.query('Ativo == "Sim"')

df_servicos

,Código,Nome,Código Tipo,Tipo,IMPLICA Despesa,Subitem,Ativo,Tipo Ativo
1,1721,SERVIÇOS DE TELEFONIA FIXA,43,Serviços de Terceiros - Pessoa Jurídica,[NATUREZA].[CÓDIGO] = 339039 ou \n[NATUREZA].[...,06,Sim,Sim
92,2462,MATERIAL DE DISTRIBUIÇÃO GRATUITA,50,"Material, Bem ou Serviço para Distribuição Gra...",[NATUREZA].[CÓDIGO] = 339032 ou\n[NATUREZA].[C...,01,Sim,Sim
127,2678,"SERVIÇOS MÉDICOS, ODONTOLÓGICOS. FARMACÊUTICO...",121,Serviços de Terceiros - Pessoa Física,[NATUREZA].[CÓDIGO] = 339036 ou [NATUREZA].[CÓ...,03,Sim,Sim
128,2679,"TREINAMENTO,RECRUTAMENTO,SELEÇÃO E SERVIÇOS AU...",121,Serviços de Terceiros - Pessoa Física,[NATUREZA].[CÓDIGO] = 339036 ou [NATUREZA].[CÓ...,10,Sim,Sim
129,2681,MANUTENÇÃO DE BENS MÓVEIS,121,Serviços de Terceiros - Pessoa Física,[NATUREZA].[CÓDIGO] = 339036 ou [NATUREZA].[CÓ...,01,Sim,Sim
...,...,...,...,...,...,...,...,...
1969,6408,DESENVOLVIMENTO DE SOFTWARE,286,Serviços de Tecnologia da Informação e Comunic...,[NATUREZA].[CÓDIGO] = 449040 ou [NATUREZA].[CÓ...,50,Sim,Sim
1970,6409,Despesas de Caráter Secreto ou Reservado de TIC,286,Serviços de Tecnologia da Informação e Comunic...,[NATUREZA].[CÓDIGO] = 339040 ou [NATUREZA].[CÓ...,95,Sim,Sim
1986,6425,SERVIÇOS DE OPERAÇÃO DE ALMOXARIFADO VIRTUAL,43,Serviços de Terceiros - Pessoa Jurídica,[NATUREZA].[CÓDIGO] = 339039,01,Sim,Sim
1987,6426,"Multas, Juros e Demais Encargos de TIC",286,Serviços de Tecnologia da Informação e Comunic...,[NATUREZA].[CÓDIGO] = 339040 ou [NATUREZA].[CÓ...,97,Sim,Sim


In [68]:
# Dividir as string na coluna 'IMPLICA Despesa' por quebra de linha
df_expanded = df_servicos['IMPLICA Despesa'].str.split('\n', expand=True)

# Reformata o DataFrame para ter cada item da String em uma nova linha
df_long_format = (
    df_servicos
    .set_index(['Código', 'Nome', 'Código Tipo', 'Tipo', 'Subitem', 'Ativo', 'Tipo Ativo'])
     .apply(lambda row: row.str.split(r'\n| ou ? | OU ', regex=True).explode())
    .reset_index()
)

# Remover linhas com valores vazios na coluna 'IMPLICA Despesa'
df_long_format = df_long_format[df_long_format['IMPLICA Despesa'].str.strip() != '']

# Função para substituir padrão específico por 33XX39
def substituir_codigo(text):
    if ('começa com 33' in text) and ('termina com 39' in text):
        return '33XX39'
    else:
        return text

# Aplicando a função em cada linha do DataFrame
df_long_format['IMPLICA Despesa'] = df_long_format['IMPLICA Despesa'].apply(substituir_codigo)


# Função para pegar os últimos seis caracteres da string com espaços removidos
def pegar_ultimos_seis_digitos(text):
    # Remover espaços em branco antes de capturar os últimos 6 caracteres
    text = text.replace(" ", "").replace("ou", "")
    if len(text) >= 6:
        return text[-6:]
    return text

# Aplicar a função em cada linha do DataFrame
df_long_format['ND'] = df_long_format['IMPLICA Despesa'].apply(pegar_ultimos_seis_digitos)

# Exibindo novo dataframe
df_long_format

,Código,Nome,Código Tipo,Tipo,Subitem,Ativo,Tipo Ativo,IMPLICA Despesa,ND
0,1721,SERVIÇOS DE TELEFONIA FIXA,43,Serviços de Terceiros - Pessoa Jurídica,06,Sim,Sim,[NATUREZA].[CÓDIGO] = 339039,339039
2,1721,SERVIÇOS DE TELEFONIA FIXA,43,Serviços de Terceiros - Pessoa Jurídica,06,Sim,Sim,[NATUREZA].[CÓDIGO] = 449039,449039
4,1721,SERVIÇOS DE TELEFONIA FIXA,43,Serviços de Terceiros - Pessoa Jurídica,06,Sim,Sim,[NATUREZA].[CÓDIGO] = 339139,339139
6,2462,MATERIAL DE DISTRIBUIÇÃO GRATUITA,50,"Material, Bem ou Serviço para Distribuição Gra...",01,Sim,Sim,[NATUREZA].[CÓDIGO] = 339032 ou,339032
7,2462,MATERIAL DE DISTRIBUIÇÃO GRATUITA,50,"Material, Bem ou Serviço para Distribuição Gra...",01,Sim,Sim,[NATUREZA].[CÓDIGO] = 339632,339632
...,...,...,...,...,...,...,...,...,...
449,6426,"Multas, Juros e Demais Encargos de TIC",286,Serviços de Tecnologia da Informação e Comunic...,97,Sim,Sim,[NATUREZA].[CÓDIGO] = 339640,339640
450,6426,"Multas, Juros e Demais Encargos de TIC",286,Serviços de Tecnologia da Informação e Comunic...,97,Sim,Sim,[NATUREZA].[CÓDIGO] = 339140,339140
451,6426,"Multas, Juros e Demais Encargos de TIC",286,Serviços de Tecnologia da Informação e Comunic...,97,Sim,Sim,[NATUREZA].[CÓDIGO] = 449140,449140
452,6426,"Multas, Juros e Demais Encargos de TIC",286,Serviços de Tecnologia da Informação e Comunic...,97,Sim,Sim,[NATUREZA].[CÓDIGO] = 449640,449640


In [69]:
df_long_format.to_excel('teste.xlsx')

In [ ]:
modalidade_96 = df_servicos[
    (df_servicos['IMPLICA Despesa'].str.contains('96', case=False, na=False)) &
    (~df_servicos['IMPLICA Despesa'].str.contains('32', case=False, na=False))
]

modalidade_96